In [2]:
## Run selenium and chrome driver to scrape data from cloudbytes.dev
import time
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import chromedriver_autoinstaller
import sqlite3

In [18]:
sample = pd.read_csv('AO3/ao3_lockwood_and_co_ao_20230616_0824.csv')
query = "SELECT * FROM fanfic"
conn = sqlite3.connect('output/ao3.db')
df = pd.read_sql_query(query, conn)
query = "SELECT * FROM authors"
df_author = pd.read_sql_query(query, conn)
query = "SELECT * FROM fandoms"
df_fandom = pd.read_sql_query(query, conn)
conn.close()
print(df_author)

    author_id                     author  joined_ao3
0     2889062                      McKat  2017-03-06
1    11032043                  ersatzist  2021-05-11
2    13764538               mayfloweruru  2022-03-22
3    14142288                  crow_blue  2022-05-12
4    16848577              tvngerine_zso  2023-03-14
..        ...                        ...         ...
663   6561073                 SweeterEnd  2020-01-07
664    574299  WindsAroundATreeLikeAVine  2014-07-26
665  14696172       Writing_random_stuff  2022-07-20
666   1711910              Amazinggrace0  2015-12-22
667  14282802              goldenjupiter  2022-05-31

[668 rows x 3 columns]


In [26]:
def get_authors(authors, fandoms):
    for x in range(len(authors)):
        print(f"getting author {x+1}/{len(authors)}")
        if pd.isnull(authors.loc[x,'author_id']):
            try:
                newlink=f"https://archiveofourown.org/users/{authors.loc[x,'author']}/profile"
                profile = requests.get(newlink, headers={
                          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
            except requests.exceptions.RequestException:
                print(f"author {authors['author'][x]} fandom is taking too long to access.")
                continue
            profile = BeautifulSoup(profile,'html.parser')
            try:
                pseuds = profile.find('dl', attrs={'class':'meta'})
                pseuds = pseuds.find_all('dd')
                joined = pseuds[1].get_text()
                id = pseuds[2].get_text()
            except:
                joined = np.nan
                id = np.nan
        
            authors.loc[x,'author_id'] = id
            authors.loc[x,'joined_ao3'] = joined
        
            time.sleep(10)
            try:
                newlink=f"https://archiveofourown.org/users/{authors.loc[x,'author']}"
                user = requests.get(newlink, headers={
                          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'}).text
            except requests.exceptions.RequestException:
                print(f"author {authors['author'][x]} profile is taking too long to access.")
                continue
            user = BeautifulSoup(user,'html.parser')
            try:
                index = user.find('ol', attrs={'class':'index group'})
                index = index.find_all('li')
                for i in range(len(index)):
                    li = index[i].get_text().split('(')
                    fandom = li[0].strip()
                    num_fic = li[-1].split(')')[0]
                    fandoms.loc[len(fandoms)] = [id, fandom, num_fic]
            except:
                print(f"author {authors['author'][x]} fandom is taking too long to access.")
            time.sleep(10)
    return authors, fandoms
authors, fandoms = get_authors(df_author, df_fandom)

getting author 1/668
getting author 2/668
getting author 3/668
getting author 4/668
getting author 5/668
getting author 6/668
getting author 7/668
getting author 8/668
getting author 9/668
getting author 10/668
getting author 11/668
getting author 12/668
getting author 13/668
getting author 14/668
getting author 15/668
getting author 16/668
getting author 17/668
getting author 18/668
getting author 19/668
author zeddmarker_ficlets (zeddmarker) fandom is taking too long to access.
getting author 20/668
getting author 21/668
getting author 22/668
getting author 23/668
author Mirroringdust (MirroringDust) fandom is taking too long to access.
getting author 24/668
getting author 25/668
getting author 26/668
getting author 27/668
getting author 28/668
getting author 29/668
getting author 30/668
getting author 31/668
getting author 32/668
getting author 33/668
getting author 34/668
getting author 35/668
author loonyloopyluna (orphan_account) fandom is taking too long to access.
getting autho

In [25]:
df_author.isnull().sum()

author_id     33
author         0
joined_ao3    33
dtype: int64

In [13]:
fanfic = df.merge(authors[['author_id', 'author']], how='left', on='author')
fanfic = fanfic.drop(columns=['author'])
conn = sqlite3.connect('output/ao3.db')
fanfic.to_sql('fanfic', conn, if_exists='replace', index=False)
authors.to_sql('authors', conn, if_exists='replace', index=False)
fandoms.to_sql('fandoms', conn, if_exists='replace', index=False)
conn.close()